In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

# Introdução - Dataset HMEQ

A base contém dados de 5.960 empréstimos concedidos por uma determinada empresa. A variável resposta, a qual tentaremos prever, será (BAD), que é binária, em que é informado se o cliente é adimplente ou não nos pagamentos do crédito obtido. 

Para cada linha da base, temos as seguintes variáveis:

* BAD - 1 = Cliente não pagou o empréstimo ou está seriamente inadimplente; 0 = Cliente com pagamentos em dia;
* CLAge	- Tempo da linha de crédito mais antiga em meses;
* CLNo - Quantidade de linhas de credito
* DebtInc - A proporção dívida / renda (DTI)  - Mede a quantidade de renda que uma pessoa ou organização gera para atender a uma dívida
* Delinq - Quantidade de linhas de crédito inadimplente
* Derog	 - Quantidade de Relatórios depreciativos
* Job - Categoria Profissional
* Loan - Montante do empréstimo solicitado
* MortDue	-  Valor Devido da Hipoteca Existente
* nInq	- Número de pedidos de crédito recentes
* Reason - DebtCon = Renegociação de dívidas; HomeImp = Melhorias em casa
* Value - Valor da Garantia Oferecida
* YoJ -	Anos de trabalho no emprego atual



# Análise Exploratória

A partir de agora, faremos algumas análises na base para entender melhor a variável resposta - BAD. Assim, tentaremos identificar possíveis correlações entre as variáveis preditoras e a variável resposta;


In [ ]:
# Carregando o Arquivo

df = pd.read_csv('/kaggle/input/hmeq-data/hmeq.csv')

df.shape

In [ ]:
# Visualizando qtde e tipos
df.info()

In [ ]:
#Visualizando os valores do DataFrame
df.head()

## Estatística Descritiva dos Empréstimos Obtidos

In [ ]:
#Empréstimos Adimplentes
df[df['BAD']==0].describe()

In [ ]:
#Emprestimos Inadimplentes
df[df['BAD']==1].describe()

* Em relação as estatíticas descritivas acima, podemos verificar:


O valor do empréstimo solicitado(LOAN), a hipoteca(MORTDUE) e a garantia subjacente são estatisticamente consistentes para os empréstimos PAGOS e que resultaram em PADRÃO. Isso sugere que essas variáveis podem não fornecer um poder de discriminação significativo para separar as duas classes.

Ainda, é possível verificar que a média de tempo no emprego (YOJ) e menor nos empréstimos inadimplentes em relação aos adimplentes. Da mesma forma, temos que o número de relatórios ruins(DEROG), o tempo de credito mais antigo (CLAGE), número de linhas de crédito inadimplentes(DLINQ)  e o número de solicitações de empréstimos, possuem influencia na variável resposta (BAD).

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
fig, axs = plt.subplots(1,2,figsize=(14,7))
sns.countplot(x='BAD',data=df,ax=axs[0])
axs[0].set_title("Frequência do Status de Pagamento")
df.BAD.value_counts().plot(x=None,y=None, kind='pie', ax=axs[1],autopct='%1.2f%%')
axs[1].set_title("Porcentagem de Pagamento")
plt.show()


## Análise Gráfica

Em uma análise inicial da base, é possível perceber que há mais de 80% de emprestimos adimplentes (BAD=0)

In [ ]:
#Tipo de emprego X Pagou emprestimo
sns.catplot(x='JOB', hue='BAD', data=df, kind='count')
#Motivo Emprestimo X Pagou emprestimo
sns.catplot(x='REASON', hue='BAD', data=df, kind='count')


## Análise Gráfica

1. É possível observar influência do tipo de trabalho da pessoal na relação com a situação do empréstimo. Assim, pode-se afirmar que a proporção de emprétimos inandimplentes é maior entre os vendedores;

2. Da mesma forma, os emprestimos solicitados para renegociação de dividas tem maior inadiplemento.

In [ ]:
# Relação PAGOU x VALOR EMPRESTIMO + REASON
sns.catplot(x='BAD', y='LOAN', hue='REASON', data=df, height=7, aspect=.8).set(title="Valor X Situação do Emprestimo e Razão")
sns.catplot(x='BAD', y='LOAN', hue='JOB', data=df, height=7, aspect=.8).set(title="Valor X Situação do Emprestimo e Trabalho")

In [ ]:
# Relação do Trabalho X Tempo de Trabalho e Situação do Emprestimo
plt.figure(figsize=(15,5))
sns.violinplot(x='JOB', y='YOJ', hue='BAD',split=True, inner="quart", data=df)  
# Relação do Trabalho X Tempo do Emprestimo Mais Antigo e Situação do Emprestimo
plt.figure(figsize=(15,5))
sns.violinplot(x='JOB', y='CLAGE', hue='BAD',split=True, inner="quart",data=df)   
# Relação do Trabalho X Valor da Hipoteca e Situação do Emprestimo
plt.figure(figsize=(15,5))
sns.violinplot(x='JOB', y='MORTDUE', hue='BAD', split=True, inner="quart",data=df)


# Tratando Valores Nulos da Base

In [ ]:
# Analisando campos Nulos
df.isna().sum()


In [ ]:
#Analisando os registros com mais da metade de valores nulos das YOJ a DEBTINC
df[df.iloc[:,6:].isnull().all(axis=1)]

In [ ]:
#Excluidno do dataFrame dados com 7 ou mais colunas com valores nulos
df = df.dropna(axis=0,thresh=df.shape[1]-6)
df[df.iloc[:,6:].isnull().all(axis=1)]

In [ ]:
# Analisando campos Nulos
df.isna().sum(), df.shape


In [ ]:
#Analisando colunas REASON do Tipo Object
df['REASON'].value_counts(),print("Nulos Campo REASON:", df['REASON'].isna().sum())


In [ ]:
#Analisando colunas JOB do Tipo Object
df['JOB'].value_counts(),print("Nulos Campo JOB:", df['JOB'].isna().sum())


In [ ]:
#Criando categoria Dummie para coluna REASON e JOB

dumies_reason=pd.get_dummies(df['REASON'],prefix='REASON')
df = df.merge(dumies_reason,left_index=True, right_index=True)
dumies_job=pd.get_dummies(df['JOB'],prefix='JOB')
df = df.merge(dumies_job,left_index=True, right_index=True)
df.head()

In [ ]:
#preenchendo campos nulos que restaram com 0
df.fillna(0,inplace=True)

In [ ]:
df.isna().sum(), df.shape

In [ ]:
# Correlação das variáveis numéricas
plt.figure(figsize= (15, 15))

sns.heatmap(df.corr(), square=True, annot=True, linewidth=0.5)


## Análise da Matrix de Correlação

Variáveis relacionadas ao histórico de crédito (DELINQ, DEROG, NINQ) tem mais correlação com a situação do empréstimo (BAD), sugerindo que essas serão as variáveis mais relevantes nos modelos de previsão.

Por outro lado, o valor do empréstimo solicitado, bem como a garantia oferecida sugerem não possuir relação com o pagamento ou não do empréstimo. 

# Modelos de Predição

A partir de agora, após a análise exploratória da base, utilizaremos alguns modelos de aprendizado de máquina para tentar prever a situação do empréstimo, adimplente (BAD=0) ou inadimplente (BAD=1). 

Assim, utilizaremos alguns dos modelos aprendidos na disciplina de ***Data Mining e Machine Learning II***: 
* *Ramdon Forest* (***RF***)
* *Gradient Boosting* (***GBM***)
* *XGBoost* (***XGB***)


In [ ]:
# Dividindo o DataFrame
from sklearn.model_selection import train_test_split

# Treino e teste
train, test = train_test_split(df, test_size=0.199, random_state=42)

# Veificando o tanho dos DataFrames
train.shape, test.shape

In [ ]:
# definindo colunas de entrada para a predição
feats = [c for c in df.columns if c not in ['BAD','JOB', 'REASON']]

In [ ]:
# Bibliotecas RandomForest
from sklearn.ensemble import RandomForestClassifier
# Bibliotecas GBM
from sklearn.ensemble import GradientBoostingClassifier
# Trabalhando com XGBoost
from xgboost import XGBClassifier

#Validação do Modelo, Acurácia
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.metrics import accuracy_score

# importando a bilbioteca para plotar o gráfico de Matriz de Confusão
import scikitplot as skplt


## Random Forest

Para obter melhor acurárica, o algoritimo de RF vai criar diversas árvores de decisão (parâmetro n_estimators) e chegar ao resultado final com base no resultado de cada árvore criada. A idéia básica é separar o conjunto de dados diversas vezes e para cada sub-conjunto treinar um novo regressor/classificador. Os diferentes regressores/classificadores irão produzir resultados diferentes, e o resultado final será determinado com base nessas regressões/classificações.

In [ ]:
#Trabalhando com Random Forest

rf = RandomForestClassifier(n_estimators=200, min_samples_split=5, oob_score=True,max_depth=4, random_state=42)
rf.fit(train[feats], train['BAD'])
rf_predict=rf.predict(test[feats])
rf_accuracy=accuracy_score(test['BAD'], rf_predict)
rf_scores = cross_val_score(rf, test[feats], rf_predict, n_jobs=-1, cv=5)
rf_model_f1=cross_validate(rf, test[feats] ,rf_predict, scoring='f1',n_jobs=-1, cv=5)
temp = pd.Series([rf_accuracy, rf_scores.mean(), rf_model_f1['test_score'].mean()], index=['ACCURACY', 'K-FOLD', 'F1'])
val_model_rf = pd.DataFrame(temp, columns=['Resultado_RF'])

In [ ]:
# Importancia das Variáveis - Modelo Random Forest
pd.Series(rf.feature_importances_, index=feats).sort_values().plot.barh()

#Matrix de Confusão - Modelo Random Forest
skplt.metrics.plot_confusion_matrix(test['BAD'] ,rf_predict, normalize=True)


In [ ]:
#Validação Modelo Random Forest

val_model_rf.head()


### Resultados Random Forest

A variável mais importante para o modelo foi a capacidade de pagamento (DEBTINC), seguida pelo número de linhas inadimplentes(DELINQ) e o valor da garantia (VALUE).

Quanto ao resultado da matriz de confusão, o modelo gerado previu de forma equivocada **1%** dos resultados para **adimplente**, ou seja, **falsos positivos**. Entretanto, ao se analisar os **falsos negativos** gerados foi de **55%** das previsões para **inadimplente**.



## Gradient Boosting
GBM é um método de boosting, construído em cima de regressores/classificadores fracos. A idéia é adicionar um regressor/classificador de cada vez, então o próximo regressor/classificador é treinado para melhorar o resultado atingido até o momento ('soma de resultados'). Ao contrário do RF, que treina cada regressor/classificador de forma independente, no GBM eles são treinados em conjunto, um ligado ao outro.

In [ ]:
# Trabalhando com GBM

gbm = GradientBoostingClassifier(n_estimators=200, learning_rate=1.0, max_depth=1, random_state=42)
gbm.fit(train[feats], train['BAD'])
gbm_predict=gbm.predict(test[feats])
gbm_accuracy = accuracy_score(test['BAD'], gbm_predict)
gbm_scores = cross_val_score(gbm, test[feats], gbm_predict, n_jobs=-1, cv=5)
gbm_model_f1=cross_validate(gbm, test[feats] ,gbm_predict, scoring='f1',n_jobs=-1, cv=5)

temp = pd.Series([gbm_accuracy, gbm_scores.mean(), gbm_model_f1['test_score'].mean()], index=['ACCURACY', 'K-FOLD', 'F1'])
val_model_gbm = pd.DataFrame(temp, columns=['Resultado_GBM'])

In [ ]:
# Importancia das Variáveis - GBM
pd.Series(gbm.feature_importances_, index=feats).sort_values().plot.barh()
#Matrix de Confusão - Modelo GBM
skplt.metrics.plot_confusion_matrix(test['BAD'] ,gbm_predict, normalize=True)


In [ ]:
#Validação Modelo GBM

val_model_gbm.head()

### Resultados **GBM**

A variável mais importante para o modelo foi a capacidade de pagamento (DEBTINC), seguida pelo número de linhas inadimplentes(DELINQ)
e o valor da garantia (VALUE).

Quanto ao resultado da matriz de confusão, o modelo gerado previu de forma equivocada em **3%** dos resultados para **adimplente**, ou seja, 
**falsos positivos**. Entretanto, ao se analisar os **falsos negativos** gerados foi de **29%** das previsões para **inadimplente**, um pouco melhor que os resultados **RF**.


## XGBoost
XGB é uma implementação específica do GBM, dita melhor e mais rápida que a implementação padrão do scikit-learn. Tanto o GBM quanto o XGB precisam de maior trabalho de interpretação dos dados e tunning do modelo.

In [ ]:
# Trabalhando com XGBoost
from xgboost import XGBClassifier
xgb = XGBClassifier(n_estimators=200, learning_rate=0.09, random_state=42)
xgb.fit(train[feats], train['BAD'])
xgb_predict=xgb.predict(test[feats])
xgb_accuracy=accuracy_score(test['BAD'], xgb_predict)
xgb_model_f1=cross_validate(xgb, test[feats] ,xgb_predict, scoring='f1',n_jobs=-1, cv=5)
xgb_scores = cross_val_score(xgb, test[feats], xgb_predict, n_jobs=-1, cv=5)

temp = pd.Series([xgb_accuracy, xgb_scores.mean(), xgb_model_f1['test_score'].mean()], index=['ACCURACY', 'K-FOLD', 'F1'])
val_model_xgb = pd.DataFrame(temp, columns=['Resultado_XGB'])

In [ ]:
# Importancia das Variáveis - XGB
pd.Series(xgb.feature_importances_, index=feats).sort_values().plot.barh()
#Matrix de Confusão - Modelo XGB
skplt.metrics.plot_confusion_matrix(test['BAD'] ,xgb_predict, normalize=True)



In [ ]:
#Validação Modelo XGB

val_model_xgb.head()

### Resultados XGB

A variável mais importante para o modelo foi a capacidade de pagamento (DEBTINC), seguida pelo número de linhas inadimplentes(DELINQ) e a quantidade de Relatórios ruins do solicitante. Diferente dos demais modelos testastos (GBM e RF), as variáveis dummies tiveram maior importância para o modelo.

Quanto ao resultado da matriz de confusão, o modelo gerado previu de forma equivocada em 3% dos resultados para adimplente, ou seja, falsos positivos. Entretanto, ao se analisar os falsos negativos gerados foi de 22% das previsões para inadimplente, o melhor resultado para os três modelos.

In [ ]:
# Compilação dos Resultados da Cross Validation dos modelos utilizados

final_result = pd.concat([val_model_rf,val_model_gbm,val_model_xgb],axis=1)

final_result.head().T

# Conclusão 

A tabela acima, demostra o desempenho dos modelos considerados no estudo. O modelos que tiveram melhores desempenhos foram o **GBM e XGB**. Contudo, o XGB foi o que menos apresentou a menor quantidade de falsos negativos, por isso a maior acurácia do modelo. Já o **GBM** teve os melhores indices de cross validation se comparado ao resultado dos demais modelos.

Assim, pelo melhor resultado na validação através do K-Fold e pelo F1, o modelo GBM teria melhor desempenho na previsibilidade dos resultados.

